In [1]:
import pandas as pd
import numpy as np
import os
import itertools
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold
from sklearn.linear_model import BayesianRidge
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error
from scipy.stats import norm

# === 資料夾設定 ===
base_dir = "Data v9"
train_dir = os.path.join(base_dir, "train")
test_dir = os.path.join(base_dir, "test")

# === 類別與連續特徵組合 ===
cat_feature_sets = {
    "vendor_te": ["vendor_te"],
    "across_state": ["across_state"],
    "vendor_across": ["vendor_te", "across_state"]
}

cont_base = ["log_volume", "log_TVP", "log_TVP/log_weight"]
distance_variants = [["log_distance"], ["log_Hdis"], ["log_Mdis"]]

cont_combinations = []
for r in range(1, len(cont_base) + 1):
    for base_combo in itertools.combinations(cont_base, r):
        for dist in distance_variants:
            cont_combinations.append(list(base_combo) + dist)

results = []
os.makedirs("best_model_plots", exist_ok=True)
os.makedirs("best_model_coefficients", exist_ok=True)
os.makedirs("all_model_results_by_ship_method", exist_ok=True)

# === 每個 ship_method 建模 ===
for file in os.listdir(train_dir):
    if not file.endswith("_train.csv"):
        continue

    method = file.replace("_train.csv", "")
    train_path = os.path.join(train_dir, file)
    test_path = os.path.join(test_dir, f"{method}_test.csv")

    if not os.path.exists(test_path):
        continue

    df_train = pd.read_csv(train_path)
    df_test = pd.read_csv(test_path)

    if len(df_train) < 100 or len(df_test) == 0:
        continue

    df_train["across_state"] = df_train["across_state"].astype(int)
    df_test["across_state"] = df_test["across_state"].astype(int)
    vendor_mean = df_train.groupby("vendor_name")["log_cost"].mean()
    global_mean = df_train["log_cost"].mean()
    df_train["vendor_te"] = df_train["vendor_name"].map(vendor_mean).fillna(global_mean)
    df_test["vendor_te"] = df_test["vendor_name"].map(vendor_mean).fillna(global_mean)

    ship_results = []

    for cat_name, cat_feats in cat_feature_sets.items():
        for cont_combo in cont_combinations:
            all_feats = list(cat_feats) + list(cont_combo)

            X_train = df_train[all_feats]
            y_train = df_train["log_cost"]
            X_test = df_test[all_feats]
            y_test = df_test["log_cost"]

            scaler = StandardScaler()
            X_train_scaled = scaler.fit_transform(X_train)
            X_test_scaled = scaler.transform(X_test)

            kf = KFold(n_splits=10, shuffle=True, random_state=42)
            train_r2s, val_r2s = [], []
            train_mses, val_mses = [], []

            for train_idx, val_idx in kf.split(X_train_scaled):
                X_tr, X_val = X_train_scaled[train_idx], X_train_scaled[val_idx]
                y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]
                model = BayesianRidge().fit(X_tr, y_tr)
                y_tr_pred = model.predict(X_tr)
                y_val_pred = model.predict(X_val)
                train_r2s.append(r2_score(y_tr, y_tr_pred))
                train_mses.append(mean_squared_error(y_tr, y_tr_pred))
                val_r2s.append(r2_score(y_val, y_val_pred))
                val_mses.append(mean_squared_error(y_val, y_val_pred))

            final_model = BayesianRidge().fit(X_train_scaled, y_train)
            y_test_pred, y_test_std = final_model.predict(X_test_scaled, return_std=True)
            test_r2 = r2_score(y_test, y_test_pred)
            test_mse = mean_squared_error(y_test, y_test_pred)

            z = norm.ppf(0.975)  # 95% interval
            pi_lower = y_test_pred - z * y_test_std
            pi_upper = y_test_pred + z * y_test_std
            pi_coverage = ((y_test >= pi_lower) & (y_test <= pi_upper)).mean()

            ship_results.append({
                "ship_method": method,
                "model": cat_name,
                "cont_features": "+".join(cont_combo),
                "Train_Adj_R2": np.mean(train_r2s),
                "Train_MSE": np.mean(train_mses),
                "Val_Adj_R2": np.mean(val_r2s),
                "Val_MSE": np.mean(val_mses),
                "Test_R2": test_r2,
                "Test_MSE": test_mse,
                "PI_coverage": round(pi_coverage, 4),
                "N_train": len(X_train),
                "N_test": len(X_test)
            })

    ship_df = pd.DataFrame(ship_results)
    ship_df.to_csv(f"all_model_results_by_ship_method/{method}_model_results.csv", index=False)
    results.extend(ship_results)

# 匯出所有模型結果
result_df = pd.DataFrame(results)
result_df.to_csv("bay_model_result_summary_full.csv", index=False)

# 自動選最佳模型
best_models = result_df[result_df["PI_coverage"] >= 0.95].sort_values("Test_MSE").groupby("ship_method").first().reset_index()
best_models.to_csv("bay_best_model_summary.csv", index=False)

print("\n✅ 模型訓練與選擇已完成，請查看：")
print("📁 all_model_results_by_ship_method/")
print("📄 model_result_summary_full.csv")
print("📄 best_model_summary.csv")


✅ 模型訓練與選擇已完成，請查看：
📁 all_model_results_by_ship_method/
📄 model_result_summary_full.csv
📄 best_model_summary.csv
